In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import json
import regex
import unidecode

In [2]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [3]:
print(df_in.shape)
print(df_ou.shape)
print(df_mi.shape)

(519520, 195)
(236210, 43)
(1650869, 107)


In [4]:
def unify_colname(col):
    return ".".join([el for el in col if el])


def evaluate(el):
    if not isinstance(el, str):
        return el
    if el[0] == "[" or el[0] == "'":
        return eval(el)
    return el


def fill_na(cell, fill=[]):
    """
    Fill elements in pd.DataFrame with `fill`.
    """
    if hasattr(cell, "__iter__"):
        if isinstance(cell, str):
            if cell == "nan":
                return fill
            return cell
        nas = []
        for el in cell:
            if el == "nan" or pd.isna(el):
                nas.append(True)
            else:
                nas.append(False)
        if all(nas):
            return fill
        return cell
    if pd.isna(cell):
        return fill
    return cell


def process_str(el: str):
    s = regex.sub(r"(^[\W]*)|([\W]*$)|([^\w:/-\s])", "", el).lower().strip()
    s = regex.sub(r"\s+", " ", s)
    return s


def process_cpv(el: str):
    return regex.sub(r"\D", "", el)

In [5]:
# Get general tender info (title, object, winner, etc)
df_in.columns = [unify_colname(c) for c in df_in.columns]
df_ou.columns = [unify_colname(c) for c in df_ou.columns]
df_mi.columns = [unify_colname(c) for c in df_mi.columns]

In [6]:
# Counter(df_in.columns.tolist()+df_ou.columns.tolist()+df_mi.columns.tolist())

In [7]:
# df[
#     "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode"
# ]

In [8]:
# print(sorted(list(set(list(df_in.columns) + list(df_ou.columns) + list(df_mi.columns))), key=len))

In [9]:
use_cols = [
    "id",
    "summary",
    "title",
    "updated",
    # "deleted_on",
    "ContractFolderStatus.ContractFolderID",
    "ContractFolderStatus.ContractFolderStatusCode",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyIdentification.ID",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyName.Name",
    "ContractFolderStatus.ProcurementProject.Name",
    "ContractFolderStatus.ProcurementProject.TypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TaxExclusiveAmount",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.DurationMeasure",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.StartDate",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.EndDate",
    # "ContractFolderStatus.TenderResult.ResultCode",
    # "ContractFolderStatus.TenderResult.ReceivedTenderQuantity",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
    "ContractFolderStatus.TenderResult.WinningParty.PartyName.Name",
    # "ContractFolderStatus.TenderResult.AwardedTenderedProject.LegalMonetaryTotal.TaxExclusiveAmount",
    # "ContractFolderStatus.TenderingProcess.ProcedureCode",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndDate",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndTime",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.Description",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod",
    # "ContractFolderStatus.ProcurementProject.SubTypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TotalAmount",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.IdentificationCode",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.Name",
    # "ContractFolderStatus.TenderResult.Description",
    # "ContractFolderStatus.TenderResult.AwardDate",
    # "ContractFolderStatus.TenderResult.StartDate",
    # "ContractFolderStatus.TenderResult.Contract.ID",
    # "ContractFolderStatus.TenderResult.Contract.IssueDate",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.CityName",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.PostalZone",
    "ContractFolderStatus.TenderingTerms.FundingProgramCode",
    "ContractFolderStatus.TenderingTerms.FundingProgram",
    "ContractFolderStatus.TenderResult.WinningParty.PartyLegalEntity.CompanyTypeCode",
]

In [10]:
df = pd.concat(
    [
        df_in[[c for c in use_cols if c in df_in.columns]],
        df_ou[[c for c in use_cols if c in df_ou.columns]],
        # df_mi[[c for c in use_cols if c in df_mi.columns]],
    ]
)
index_names = df.index.names
df.reset_index(inplace=True)
df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
# df.drop(index_names, inplace=True, axis=1)
df.set_index("identifier", inplace=True)
df = df.applymap(fill_na, fill=np.nan)
df.shape

(755730, 24)

In [11]:
# with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
#     gencat = pd.json_normalize(json.load(f))

In [12]:
# tend_cat2 = pd.read_csv(r"C:\Users\josea\Downloads\empresas2.csv")
tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_24056\886581414.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [13]:
rename_columns = {
    "title": "title",
    "summary": "summary",
    "ContractFolderStatus.ContractFolderID": "id",
    "ContractFolderStatus.ProcurementProject.Name": "project",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID": "winner",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity": "location",
    "denominacio": "title",
    "objecte_contracte": "summary",
    "codi_expedient": "id",
    "codi_cpv": "cpv",
}

In [14]:
tender_filt = (
    df[
        [
            "title",
            "summary",
            "ContractFolderStatus.ContractFolderID",
            # "ContractFolderStatus.ProcurementProject.Name",
            # "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
            "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
            "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
        ]
    ]
    .rename(columns=rename_columns)
    .dropna(how="all")
    .explode("cpv")
    # .explode("winner")
)
tender_filt["id"] = tender_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
# tender_filt["winner"] = tender_filt["winner"].apply(evaluate)
tender_filt["cpv"] = tender_filt["cpv"].apply(evaluate)
tender_filt = tender_filt.explode("cpv")  # .explode("winner")
tender_filt["cpv"] = tender_filt["cpv"].astype(str).apply(process_cpv)
tender_filt["location"] = tender_filt["location"].apply(
    lambda x: unidecode.unidecode(x.lower()) if not pd.isna(x) else np.nan
)
tender_filt = (
    tender_filt.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
tender_filt["cpv_div"] = tender_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)

In [15]:
agg_tender = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .groupby(["id"])
    .agg(list)
)
valid_agg_tender = agg_tender[agg_tender["title"].apply(lambda x: len(set(x)) == 1)]
valid_agg_tender = valid_agg_tender.applymap(
    lambda x: Counter(x).most_common()[0][0]
).reset_index()
print(valid_agg_tender.shape)
display(valid_agg_tender.head())

(245975, 6)


,id,title,summary,cpv,location,cpv_div
0,0-2020-5-2020,rehabilitació de casa de poblet,id licitación: 0/2020-5/2020 órgano de contrat...,454541000,nan,45
1,00-0000,licitació de prova sobre digital 00/0000 - pro...,id licitación: 00/0000 órgano de contratación:...,791000000,nan,79
2,00-2019,comunicació de contractació anual programada e...,id licitación: 00/2019 órgano de contratación:...,853200000,nan,85
3,00-d-12-0000572,servicio de mantenimiento del sistema de alime...,id licitación: 00/d/12/0000572 órgano de contr...,505320000,nan,50
4,00-d-12-421,servicio de mantenimiento de la licencia dynat...,id licitación: 00/d/12/421 órgano de contratac...,48700000,nan,48


In [16]:
# tender_filt["id_clean"] = tender_filt["id"].apply(
#     lambda x: unidecode.unidecode(regex.sub(r"\W", "", x)) if not pd.isna(x) else np.nan
# )
# agg_tender2 = (
#     tender_filt[
#         tender_filt["location"].str.contains(
#             "|".join(
#                 [
#                     "nan",
#                     "espana",
#                     "cataluna",
#                     "catalunya",
#                     "barcelona",
#                     "tarragona",
#                     "girona",
#                     "gerona",
#                     "lleida",
#                     "lerida",
#                 ]
#             )
#         )
#     ]
#     .groupby(["id_clean"])
#     .agg(list)
# )
# valid_agg_tender2 = agg_tender2[agg_tender2["title"].apply(lambda x: len(set(x)) == 1)]
# valid_agg_tender2 = valid_agg_tender2.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()
# print(valid_agg_tender.shape)
# display(valid_agg_tender.head())

In [17]:
# print(valid_agg_tender2.shape)
# display(valid_agg_tender2.head())

In [19]:
# Clean tenders cat
tend_cat_filt = (
    tend_cat[
        [
            "denominacio",
            "objecte_contracte",
            "codi_expedient",
            "codi_cpv",
        ]
    ]
    .rename(columns=rename_columns)
    .astype(str)
    .applymap(process_str)
)
tend_cat_filt["id"] = tend_cat_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
tend_cat_filt["cpv"] = tend_cat_filt["cpv"].astype(str).apply(process_cpv)
tend_cat_filt = (
    tend_cat_filt.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
tend_cat_filt["cpv_div"] = tend_cat_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)

In [20]:
tend_cat_filt.head()

,title,summary,id,cpv,cpv_div
0,acord marc per al subministrament i installaci...,subministrament i installació de mobiliari de ...,ccs-2022-6,391000003,39
1,acord marc per al subministrament i installaci...,subministrament i installació de mobiliari de ...,ccs-2022-6,391000003,39
2,acord marc per al subministrament i installaci...,subministrament i installació de mobiliari de ...,ccs-2022-6,391000003,39
3,publicació agregada contractes menors dacc 1r ...,compra de dues minicabines de seguretat biològica,ag-2023-662,380000005,38
4,publicació agregada contractes menors dacc 1r ...,redacció pla dautoprotecció de lembassament de...,ag-2023-663,713560008,71


In [21]:
# tend_cat_filt[tend_cat_filt["id"]=="ccs-2022-6"]
# c = []
# for el in tend_cat[:3].apply(lambda x: Counter(x), axis=0):
#     c.append(3 not in el.values())
# tend_cat[:3][tend_cat.columns[c]]

In [22]:
# Aggregate by ID
valid_tend_cat = tend_cat_filt.groupby("id").agg(list)
valid_tend_cat = valid_tend_cat[
    valid_tend_cat["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat = valid_tend_cat.applymap(
    lambda x: Counter(x).most_common()[0][0]
).reset_index()
print(len(valid_tend_cat))
display(valid_tend_cat.head())

227651


,id,title,summary,cpv,cpv_div
0,0-2020-5-2020,rehabilitació de casa de poblet,rehabilitació de casa de poblet,454541005,45
1,0-s-04-2023,gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,79
2,00-0000,licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,791000005,79
3,000,concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,NaN
4,000-expedient-2020-034,000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,926220007,92


In [23]:
tend_cat[tend_cat["denominacio"].str.lower() == "rehabilitació de casa de poblet"][
    tend_cat.columns[10:]
]

,codi_expedient,tipus_contracte,procediment,fase_publicacio,denominacio,objecte_contracte,valor_estimat_contracte,codi_nuts,lloc_execucio,durada_contracte,...,tipus_empresa,url_json_futura,url_json_cpm,url_json_previ,url_json_licitacio,url_json_avaluacio,url_json_adjudicacio,url_json_formalitzacio,url_json_anulacio,url_json_agregada
153086,0/2020-5/2020,Obres,Obert Simplificat,Adjudicació,Rehabilitació de Casa de Poblet,Rehabilitació de Casa de Poblet,167714.63,ES513,Lleida,15/12/2021 a 15/04/2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
valid_tend_cat

,id,title,summary,cpv,cpv_div
0,0-2020-5-2020,rehabilitació de casa de poblet,rehabilitació de casa de poblet,454541005,45
1,0-s-04-2023,gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,79
2,00-0000,licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,791000005,79
3,000,concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,NaN
4,000-expedient-2020-034,000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,926220007,92
...,...,...,...,...,...
227646,z8489,subministrament i installació de lescomesa de ...,subministrament i installació de lescomesa de ...,515000007,51
227647,z8655,obres dinstallació duna planta fotovoltaica pe...,obres dinstallació duna planta fotovoltaica pe...,317123319,31
227648,z8672,obres dampliació de la xarxa del tub verd per ...,obres dampliació de la xarxa del tub verd per ...,452200005,45
227649,zls23-0084,2023 1r trimestre contractes menors associació...,traduccions,795300008,79


In [25]:
tend_cat_filt.groupby("id").agg(list)

,title,summary,cpv,cpv_div
id,,,,
0-2020-5-2020,[rehabilitació de casa de poblet],[rehabilitació de casa de poblet],[454541005],[45]
0-s-04-2023,[gestió duna installació itinerant sobre la pr...,[gestió duna installació itinerant que parli d...,[794210001],[79]
00-0000,[licitació de prova sobre digital],[licitació de prova sobre digital 00/0000 - pr...,[791000005],[79]
000,[concessió dús privatiu per a la installació d...,[concessió dús privatiu per a la installació d...,[nan],[nan]
000-expedient-2020-034,[000 expedient 2020 034 - licitació de la cont...,[en el marc duna creixent proliferació quant a...,[926220007],[92]
...,...,...,...,...
z8489,[subministrament i installació de lescomesa de...,[subministrament i installació de lescomesa de...,[515000007],[51]
z8655,[obres dinstallació duna planta fotovoltaica p...,[obres dinstallació duna planta fotovoltaica p...,[317123319],[31]
z8672,[obres dampliació de la xarxa del tub verd per...,[obres dampliació de la xarxa del tub verd per...,[452200005],[45]


# TODO
- Hacer match con budget

In [40]:
# Match tenders by ID
matched_id = pd.merge(
    valid_agg_tender,
    valid_tend_cat,
    how="inner",
    left_on=["id"],
    right_on=["id"],
)
matched_id = matched_id[
    [
        "id",
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
    ]
]
title = []
cpv_div = []
for tx, ty, cx, cy in matched_id[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    title.append(Counter(set([tx, ty])))
    cpv_div.append(Counter(set([cx, cy])))

matched_id["title"] = title
matched_id["cpv_div"] = cpv_div

matched_id = matched_id[["id", "title", "cpv_div"]]
print(len(matched_id))
print(sum(matched_id["title"].apply(lambda x: sum(x.values()))))
display(matched_id.head())

83775
140485


,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000-expedient-2020-034,{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000-expedient-2021-266,{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa de subministrament dai...,{'45': 1}


In [41]:
# Match tenders by title and CPV
matched_aux = pd.merge(
    valid_agg_tender,
    valid_tend_cat,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
)
matched_aux = matched_aux[
    [
        "id_x",
        "id_y",
        "title",
        "cpv_div",
    ]
]
matched_aux = matched_aux[matched_aux["id_x"] != matched_aux["id_y"]]

# # ERROR
# matched_aux[
#     (~matched_aux["id_x"].isin(matched_id["id"]))
#     & (~matched_aux["id_y"].isin(matched_id["id"]))
# ]


m_x = matched_aux.loc[
    ~matched_aux["id_x"].isin(matched_id["id"]), ["id_x", "title", "cpv_div"]
].rename({"id_x": "id"}, axis=1)
m_y = matched_aux.loc[
    ~matched_aux["id_y"].isin(matched_id["id"]), ["id_y", "title", "cpv_div"]
].rename({"id_y": "id"}, axis=1)
matched_aux = pd.concat([m_x, m_y]).groupby("id").agg(Counter).reset_index()

# title = []
# cpv_div = []
# for tx, ty, cx, cy in matched_aux[["title_x", "title_y", "cpv_div_x", "cpv_div_y"]].values:
#     title.append(Counter(set([tx, ty])-set([np.nan])))
#     cpv_div.append(Counter(set([cx, cy])-set([np.nan])))
#     # t = dict()
#     # c = dict()
#     # t.update(tx if not pd.isna(tx) else dict())
#     # t.update(ty if not pd.isna(ty) else dict())
#     # c.update(cx if not pd.isna(cx) else dict())
#     # c.update(cy if not pd.isna(cy) else dict())
#     # title.append(t)
#     # cpv_div.append(c)

# matched_aux["title"] = title
# matched_aux["cpv_div"] = cpv_div

matched_aux = matched_aux[["id", "title", "cpv_div"]]
print(len(matched_aux))
print(sum(matched_aux["title"].apply(lambda x: sum(x.values()))))
display(matched_aux.head())

2973
9836


,id,title,cpv_div
0,001-22,{'contractació dun servei que inclogui 3 perso...,{'72': 1}
1,002-20,{'contractació dun servei que inclogui 3 perso...,{'72': 1}
2,003-23,{'contractació dun servei que inclogui 3 perso...,{'72': 2}
3,0047,{'servei de manteniment preventiu i correctiu ...,{'50': 1}
4,006-19,{'maquetació de la revista informat versió en ...,{'79': 1}


In [42]:
# Combine both
matched_final = pd.merge(
    matched_id,
    matched_aux,
    how="outer",
    left_on="id",
    right_on="id",
)
title = []
cpv_div = []
for tx, ty, cx, cy in matched_final[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    t = dict()
    c = dict()
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    title.append(t)
    cpv_div.append(c)

matched_final["title"] = title
matched_final["cpv_div"] = cpv_div

matched_final = matched_final[["id", "title", "cpv_div"]]
print(len(matched_final))
print(sum(matched_final["title"].apply(lambda x: sum(x.values()))))
display(matched_final.head())

86748
150321


,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000-expedient-2020-034,{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000-expedient-2021-266,{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa de subministrament dai...,{'45': 1}


In [53]:
# Unique indices (by id)
_tot_id = len(matched_id) / len(valid_tend_cat)
# Total elements (taking into account some values appear multiple times)
_tot_id_sum = matched_id["title"].apply(lambda x: sum(x.values())).sum() / len(
    valid_tend_cat
)

# Unique indices (by cpv and title)
_tot_ti = len(matched_aux) / len(valid_tend_cat)
# Total elements (taking into account some values appear multiple times)
_tot_ti_sum = matched_aux["title"].apply(lambda x: sum(x.values())).sum() / len(
    valid_tend_cat
)

# Unique aggregated
_tot = len(matched_final) / len(valid_tend_cat)
# Total elements (taking into account some values appear multiple times)
_tot_sum = matched_final["title"].apply(lambda x: sum(x.values())).sum() / len(
    valid_tend_cat
)

print(f"{'Unique indices (by id): ':<40}{_tot_id:.5f}")
print(f"{'Total elements (by id): ':<40}{_tot_id_sum:.5f}")
print()
print(f"{'Unique indices (by cpv and title): ':<40}{_tot_ti:.5f}")
print(f"{'Total elements (by cpv and title): ':<40}{_tot_ti_sum:.5f}")
print()
print(f"{'Unique indices (aggregated): ':<40}{_tot:.5f}")
print(f"{'Total elements (aggregated): ':<40}{_tot_sum:.5f}")

Unique indices (by id):                 0.36800
Total elements (by id):                 0.61711

Unique indices (by cpv and title):      0.01306
Total elements (by cpv and title):      0.04321

Unique indices (aggregated):            0.38106
Total elements (aggregated):            0.66031
